In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import one_hot
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import numpy as np

from dataloader import train_dataloader, val_dataloader, SubjectDataset, train_dataset
from model import BRNN, OneDConvNet
from torch.utils.data import Dataset, DataLoader

In [16]:
from torch.nn import functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 6
sequence_length = 335413
num_layers = 2
hidden_size = 125
num_classes = 4
learning_rate = 0.0001
batch_size = 128
num_epochs = 20

In [23]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def train_step(X, y, model, optimizer, criterion):
    y = F.one_hot(y, 4).float()
    y_pred = model(X)
    predicted_classes = torch.argmax(y_pred.detach(), dim=1)

    loss = criterion(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    corrects = accuracy_fn(y.argmax(dim=1), y_pred.argmax(dim=1))

    return loss.item(), corrects

def val_step(X, y, model, criterion):

    with torch.no_grad():
        y = F.one_hot(y, 4).float()
        y_pred = model(X)
        predicted_classes = torch.argmax(y_pred.detach(), dim=1)
        loss = criterion(y_pred, y)
        corrects = accuracy_fn(y.argmax(dim=1), y_pred.argmax(dim=1))

    return loss.item(), corrects, predicted_classes.detach().cpu().numpy()

In [9]:
best_val_loss = float("inf")
stats = {
  "acc_x": {
    "min": -4.095068,
    "max": 4.42501
  },
  "acc_y": {
    "min": -0.27057,
    "max": 10.626847
  },
  "acc_z": {
    "min": -2.556078,
    "max": 11.227555
  },
  "gyro_x": {
    "min": -2.133614,
    "max": 0.72327
  },
  "gyro_y": {
    "min": -1.113133,
    "max": 1.058619
  },
  "gyro_z": {
    "min": -0.771591,
    "max": 0.606588
  }
}
min = np.array([v["min"] for k, v in stats.items()])
max = np.array([v["max"] for k, v in stats.items()])

min = torch.from_numpy(min).float()
min = torch.unsqueeze((torch.unsqueeze(min, 0)), -1)
min = min.to(device)
max = torch.from_numpy(max).float().to(device)
max = torch.unsqueeze((torch.unsqueeze(max, 0)), -1)
max = max.to(device)

In [4]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [24]:
model = OneDConvNet(input_size, num_classes, 16, 40).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)

In [25]:
for epoch in range(num_epochs):
    # Train for "n" number of iterations
    running_loss = 0.
    running_acc = 0.
    for iteration, (X, y) in enumerate(train_dataloader):

        X = X.float().to(device)
        # Normalize
        X = (X - min) / (max - min)
        # X = torch.transpose(X, 1, 2)
        y = y.view(X.size(0)).to(device)

        loss, corrects = train_step(X, y, model, optimizer, criterion)

        # Running metrics
        running_loss = running_loss + loss 
        running_acc = running_acc + corrects

        

    train_loss = running_loss / len(train_dataloader)
    train_acc = running_acc / len(train_dataloader)

    # Validate
    running_val_loss = 0.
    running_val_acc = 0.
    for step, (X, y) in enumerate(val_dataloader):

        X = X.float().to(device)
        X = (X - min) / (max - min)
        # X = torch.transpose(X, 1, 2)
        y = y.view(X.size(0)).to(device)

        loss, corrects, predicted_classes = val_step(X, y, model, criterion)
        # Running metrics
        running_val_loss = running_val_loss + loss
        running_val_acc = running_val_acc + corrects

    val_loss = running_val_loss / len(val_dataloader)
    val_acc = running_val_acc / len(val_dataloader)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        # Checkpoint model
        path = "checkpoint_model_lstm_filtered_run_1.pth"
        print(f"Saving model to {path}")
        torch.save(model.state_dict(), path)
        best_val_loss = val_loss

    print(f"Epoch: {epoch} | train_loss {train_loss} | train_acc: {train_acc} | val_loss: {val_loss} | val_acc: {val_acc}")

Epoch: 0 | train_loss 0.39284455961221226 | train_acc: 82.5146103896104 | val_loss: 0.8730780971993646 | val_acc: 67.84847122302158
Epoch: 1 | train_loss 0.2861367619269854 | train_acc: 87.40868506493507 | val_loss: 0.8600324131602006 | val_acc: 70.24168165467626
Epoch: 2 | train_loss 0.26052614524767 | train_acc: 88.80397727272727 | val_loss: 0.9935020611440535 | val_acc: 69.76618705035972
Epoch 00004: reducing learning rate of group 0 to 5.0000e-05.
Epoch: 3 | train_loss 0.24597075807196753 | train_acc: 89.51866883116882 | val_loss: 0.9790898859929695 | val_acc: 67.74280575539568
Epoch: 4 | train_loss 0.22528779742392627 | train_acc: 90.62012987012987 | val_loss: 1.039667752235056 | val_acc: 65.52720323741008
Epoch 00006: reducing learning rate of group 0 to 2.5000e-05.
Epoch: 5 | train_loss 0.21766162261947408 | train_acc: 90.95048701298701 | val_loss: 1.0091192838099363 | val_acc: 69.32329136690647
Epoch: 6 | train_loss 0.20958074301094204 | train_acc: 91.32305194805195 | val_loss:

In [69]:
import pandas as pd
FILENAME_TEMPLATE = "subject_{}_{}__y.csv"

In [70]:
save_dir = "terrain-identification/predictions/C3.2/LSTM"

In [65]:
test_data_path = "terrain-identification/data/TestData/window_3"
split_ids = ["009_01", "010_01", "011_01", "012_01"]
batch_size = 128
model.eval()
for id in split_ids:

    test_dataset = SubjectDataset(
        test_data_path, 
        [id]
    )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    

    
    

    output = []

    for (X, y) in test_dataloader:

        X = X.float().to(device)
        X = (X - min) / (max - min)

        X = torch.transpose(X, 1, 2)
        if X.shape[0] != 128:
            X = torch.cat((X, torch.zeros((128-X.shape[0], 120, 6)).to(device)), dim=0)
        y_pred = model(X)
        

       
        predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()

        output.append(predicted_classes)

    _output = np.concatenate(output, axis=0)

    df = pd.DataFrame({"label": _output})

    subject_id, session_id = id.split("_")

    filename = FILENAME_TEMPLATE.format(subject_id, session_id)
    df.to_csv(os.path.join(save_dir, filename), header=False, index=False)